# Generación de texto creativo con IA - Poesía

### Librerías

In [ ]:
from transformers import XGLMTokenizerFast, XGLMForCausalLM
from transformers import DataCollatorWithPadding, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from tokenizers import AddedToken
from datasets import load_metric
from datasets import load_dataset
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb

### Carga de modelo, tokenizador y ajuste de tokenizador

Se carga la versión 1.7B del modelo XGLM, indicándole que el dispositivo que se usará es la GPU (CUDA: Compute Unified Device Architecture de Nvidia). Adicionalmente se le indica la ruta de la máquina local donde quedará alojado el modelo modelo descargado. Se agrega el token especial '\n' que hace la función de salto de línea, el cual no está en el vocabulario del tokenizador. Adicionalmente se redimensiona el tamaño del token embedding.

In [ ]:
tokenizer = XGLMTokenizer.from_pretrained("facebook/xglm-1.7B")
model = XGLMForCausalLM.from_pretrained("facebook/xglm-1.7B", device_map="cuda", load_in_8bit=True, cache_dir= 'I:/transformers')

In [ ]:
special_tokens = {
    'additional_special_tokens': [AddedToken('\n')]
}

# Modifica los tokens especiales del tokenizador
tokenizer.add_special_tokens(special_tokens)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

### Adquisición de conjunto de datos de poesía, limpieza y pre-procesado

Se carga el conjunto de datos que contiene las poesías. Se eligen únicamente los textos en español y se pre-procesa el conjunto de datos, agregándole los tokens representarán el inicio y final de cada poesía. Despues, se tokeniza el conjunto de datos, truncandolos a 512 máximo de longitud y se realiza padding para rellenar aquellos textos que tenga una longitud menor. Finalmente se declara el data collator que se usará en el entrenamiento. 

In [ ]:
# Cargar el conjunto de datos
dataset = load_dataset("linhd-postdata/poesias")

# Acceder a la información del conjunto de datos
print(dataset)

In [ ]:
dataset_spanish = dataset.filter(lambda example: example['language'] == 'es')

In [ ]:
def format_ds(example):
  example["text"] = "<s>" + example['text'] + "</s>"
  return example

In [ ]:
dataset_spanish = dataset_spanish.map(format_ds)

In [ ]:
tokenized_data = dataset_spanish.map(
    lambda example: tokenizer(example["text"], max_length=512, truncation=True),
    batched=True,
)


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,  mlm=False)

### Entrenamiento y evaluación por época

Se itera sobre los parámetros del modelo y los congela para que no se actualicen durante el entrenamiento y se observa la cantidad final de parámetros del modelo.

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Despues se usa la biblioteca peft para configurar un modelo LoRA (Low-Rank Adaptation of Large Language Models) con una configuración específica definida por LoraConfig. Después, se imprime el número de parámetros entrenables y el número total de parámetros del modelo.

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, # num de variabkle sindep a tener cada matriz para modificar comportamiento, se
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['embed_tokens', 'lm_head']
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

En training_args se definen los argumentos principales de entrenamiento del modelo y esto alimenta el trainer, que se usará para entrenar el modelo y realizar la evaluación por época.

In [ ]:
training_args = TrainingArguments(
    output_dir='I:/transformers/xglmmed3',
    learning_rate= 0.0001, 
    save_steps = 1000000,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.001,
    gradient_accumulation_steps=2,
    do_eval = True,
    fp16=True,
    evaluation_strategy  = 'epoch',
    eval_steps = 1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

#### Guardado del modelo y métricas de rendimiento

El modelo entrenado y las métricas de rendimiento se guardan en el equipo local.

In [ ]:

model.save_pretrained("C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/modelos_generadores/xglm_1_7b_poesia")

In [ ]:
logh = trainer.state.log_history

In [ ]:
extracted_data = []

# Itera sobre cada diccionario en la lista 'data'
for entry in logh:
    # Extrae las claves 'loss', 'step' y 'eval_loss' de cada diccionario
    extracted_entry = {'loss': entry.get('loss', None),
                       'step': entry.get('step', None),
                       'eval_loss': entry.get('eval_loss', None)}

    # Agrega el diccionario extraído a la nueva lista
    extracted_data.append(extracted_entry)



In [ ]:
# Crea un DataFrame de pandas con los datos extraídos
df = pd.DataFrame(extracted_data)
csv_file_path = 'C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/metricas_modelo/loss_xglm_1_7b.csv'
df.to_csv(csv_file_path, index=False)

### Despliegue

#### Carga de modelo, tokenizador y ajuste de tokenizador

Se carga el modelo almacenado, así como el tokenizador original que se vuelve a ajustar para incluir el caracter '\n'.

In [ ]:
ruta_modelo = "C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/modelos_generadores/xglm_1_7b_poesia"
# XGLMTokenizer, XGLMForCausalLM
# Cargar el tokenizador y el modelo
model = XGLMForCausalLM.from_pretrained(ruta_modelo, device_map="cuda", load_in_8bit=True)
tokenizer = XGLMTokenizerFast.from_pretrained("facebook/xglm-1.7B")

In [ ]:
special_tokens = {
    'additional_special_tokens': [AddedToken('\n')]
}

# Modifica los tokens especiales del tokenizador
tokenizer.add_special_tokens(special_tokens)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

#### Función generadora de poesía

A partir del modelo cargado, se crea esta función generadora de texto, la cual se alimenta de una entrada textual y genera la poesía. Se declara el argumento do_sample a True para que genere diferentes poesías con la misma entrada. El argumento temperatura controla la aleatoriedad de la generación. Valores bajos producen texto mas conservador eligiendo palabras mas probables. Valores altos produce mayor variabilidad y creatividad en la generación. Un valor en 0.95 garantiza variabilidad y creatividad.

In [ ]:
def generar_texto(Texto):
    input_ids = tokenizer.encode(Texto, return_tensors="pt").to('cuda')

    # Generar texto condicionalmente
    output = model.generate(input_ids, max_length=250, min_length=100,pad_token_id=tokenizer.eos_token_id,
                        #repetition_penalty = 1.2, num_beams=1, 
                        #num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, 
                        temperature=0.95, do_sample=True,)

# Decodificar y mostrar el texto generado
    texto_generado = tokenizer.decode(output[0], skip_special_tokens=False)

    
    #texto_generado = "Texto generado por el modelo..." # Aquí deberías llamar a tu modelo generativo para generar texto
    
    return texto_generado

#### Interfaz web para generar poesía con Gradio

Usando la libreria Gradio, a partir de la función anterior, se genera una interfaz web que permite ingresar un texto inicial y así generar poesía a partir de dicho texto. La interfaz permite interactuar con ella en el presente notebook o mediante un link web.

In [ ]:
interfaz = gr.Interface(
    fn=generar_texto,
    inputs="text",
    outputs="text",
    title="Generador de Poesía",
    description="Introduce un texto inicial y genera poesía",
    allow_flagging = 'never',
    clear_btn = 'Nueva Poesía',
    submit_btn = 'Crea Poesía'

)

Se lanza la interfaz web por medio de launch().

In [ ]:
interfaz.launch(share=True)

Se cierra la interfaz web por medio de close().

In [ ]:
interfaz.close()